In [1]:
import boto3
import botocore
import datetime
import csv

# Replace these variables with your own values
# Initialize the S3 client
s3_client = boto3.client('s3')

# List all S3 buckets
response = s3_client.list_buckets()

# Extract bucket names from the response
bucket_names = [bucket['Name'] for bucket in response['Buckets']]

# Initialize the CloudWatch client for the specified AWS region
cloudwatch_client = boto3.client('cloudwatch')

# Set start and end dates for February
start_time = datetime.datetime(2024, 2, 1)
end_time = datetime.datetime(2024, 3, 1) - datetime.timedelta(seconds=1) 


with open('bucket_metrics.csv', 'w', newline='') as csvfile:
    # Initialize CSV writer
    writer = csv.writer(csvfile)
    # Write header row
    writer.writerow(['Bucket Name', 'Size (GB)','Size (MB)', 'Cost($)','File Count','No_of_requests'])

    for bucket_name in bucket_names:
        try:
            #count of s3 bucket
            bucket_objects = s3_client.list_objects_v2(Bucket=bucket_name)
    
            # Query CloudWatch for S3 bucket size metrics
            response_1 = cloudwatch_client.get_metric_statistics(
                Namespace='AWS/S3',
                MetricName='BucketSizeBytes',
                Dimensions=[
                    {'Name': 'BucketName', 'Value': bucket_name},
                    {'Name': 'StorageType', 'Value': 'StandardStorage'}
                ],
                StartTime=start_time,
                EndTime=end_time,
                Statistics=['Average'],
                Period=3600
            )
        
            # Extract and display the bucket size if available
            size_in_bytes = response_1['Datapoints'][-1]['Average'] if response_1['Datapoints'] else None

            # Query CloudWatch for S3 bucket size metrics
            response_2 = cloudwatch_client.get_metric_statistics(
                Namespace='AWS/S3',
                MetricName='NumberOfObjects',  # Modified MetricName
                Dimensions=[
                    {'Name': 'BucketName', 'Value': bucket_name},
                    {'Name': 'StorageType', 'Value': 'AllStorageTypes'}  # Use 'AllStorageTypes' for NumberOfObjects
                ],
                StartTime=start_time,
                EndTime=end_time,
                Statistics=['Average'],
                Period=3600
            )
            
            # Extract and display the number of objects if available
            num_objects = response_2['Datapoints'][-1]['Average'] if response_2['Datapoints'] else None

            if num_objects is not None:
                print(int(num_objects))
        
            if size_in_bytes:
                size_in_megabytes = size_in_bytes / (1024 * 1024 )  # Convert bytes to megabytes
                print(f"Bucket Size: {size_in_megabytes:.2f} MB")
                size_in_gigabytes = size_in_bytes / (1024 * 1024 * 1024)  
                print(f"Bucket Size: {size_in_gigabytes:.2f} GB")
                cost = size_in_bytes * 0.000000001 * 0.023
                print(f" Cost: ${cost:.5f}")
                print("File Count:",num_objects)
                writer.writerow([bucket_name,f"{size_in_gigabytes:.2f}", f"{size_in_megabytes:.2f}", f"{cost:.5f}",int(num_objects)]) # Write row to CSV
            else:
                print("No metric data points available for bucket size.")
        
        except botocore.exceptions.ClientError as error:
            print(f"Error: {str(error)}")

No metric data points available for bucket size.
No metric data points available for bucket size.
1
Bucket Size: 11557.42 MB
Bucket Size: 11.29 GB
 Cost: $0.27873
File Count: 1.0
7109
Bucket Size: 35311.42 MB
Bucket Size: 34.48 GB
 Cost: $0.85161
File Count: 7109.0
9
Bucket Size: 85.19 MB
Bucket Size: 0.08 GB
 Cost: $0.00205
File Count: 9.0
5997
Bucket Size: 391.71 MB
Bucket Size: 0.38 GB
 Cost: $0.00945
File Count: 5997.0
70
Bucket Size: 12.10 MB
Bucket Size: 0.01 GB
 Cost: $0.00029
File Count: 70.0
148
Bucket Size: 133.55 MB
Bucket Size: 0.13 GB
 Cost: $0.00322
File Count: 148.0
90
Bucket Size: 194.95 MB
Bucket Size: 0.19 GB
 Cost: $0.00470
File Count: 90.0
25
Bucket Size: 19.09 MB
Bucket Size: 0.02 GB
 Cost: $0.00046
File Count: 25.0
3
Bucket Size: 0.01 MB
Bucket Size: 0.00 GB
 Cost: $0.00000
File Count: 3.0
49
Bucket Size: 0.72 MB
Bucket Size: 0.00 GB
 Cost: $0.00002
File Count: 49.0
898
Bucket Size: 3744.15 MB
Bucket Size: 3.66 GB
 Cost: $0.09030
File Count: 898.0
5
Bucket Size: 0

In [3]:
import pandas as pd
# Read the CSV file into a DataFrame
df = pd.read_csv('bucket_metrics.csv')
sorted_df = df.sort_values(by='Cost($)',ascending=False)


sorted_df
sorted_df.to_csv('s3_cost_details')